# Implémentation d’un algorithme de points intérieurs 

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Introduction
Les algorithmes de points intérieurs sont des algorithmes qui permettent d’optimiser des problèmes linéaires et non linéaires avec contraintes. Ils sont dérivés de la méthode des points extérieures (ou des ellipses) qui avait pour but d’optimiser des problèmes non linéaires avec contraintes, mais qui avait une trop grande complexité en espace(bits pour stocker).
Den Hertog (1994) a classé les méthodes de points intérieurs en quatre catégories :
- Méthodes de chemin central
- Méthodes affines et mise à l’échelle
- Méthodes projectives avec potentiel
- Méthodes affines avec potentiel


Au fil de ce notebook, nous porterons notre attention plus particulièrement sur les
méthodes de chemin central ou critique, et plus précisément de barrière logarithmique, et son
implémentation en langage Python.

In [2]:
from decimal import *
def deriveepolynome(liste):
    d = [Decimal('0')]*(len(liste)-1)
    for i in range(len(d)):
        d[i] += Decimal(int(liste[i]) * (len(liste) - i - 1))
    return d
def deriveepartielle(liste):
    dp = []
    for j in range(len(liste)):
        dp.append(deriveepolynome(liste[j]))
    return dp

In [3]:
def resultat(v, polynome):
    r= Decimal(0)
    deg=(len(polynome))
    for k in range(int(deg)):
        if v!=0:
            r += polynome[k]*(v**(deg-k-1))
        else:
            if deg-k-1==0:
                r += polynome[k]
            else:
                r+=0
    return r

In [4]:
def somme(x, contrainte):
    s=Decimal('0')
    for i in range(len(x)):
        s+= Decimal(resultat(x[i], contrainte[i]))
    return s

Le principe général de cette méthode revient à transformer le programme contraint en un programme sans contraintes, ici à l’aide d’un fonction logarithmique, qui par
la définition du logarithme, défini un nouveau domaine de définition pour la fonction objectif à minimiser. L’existence d’une solution au problème repose essentiellement
sur un critère : l’existence au moins d’un point appartenant au domaine de définition de la fonction objectif obtenue. Cette fonction ext dite donc pénalisée. Dans ce but,
nous réécrivons l’ensemble de toutes nos contraintes sous formes d’inéquations de supériorité (le domaine de définition du log étant R∗+ ).

In [49]:
def contraintelineaire(x, fonctob, listcont):
    u=Decimal(0.01)
    d=deriveepartielle(fonctob)
    for j in range(len(listcont)):
        s=listcont[j][len(x)]
        for i in range(len(x)):
            s+=x[i]*listcont[j][i]
        if s<=0:
            for k in range(len(x)):
                if (listcont[j][k]!=0):
                    d[k]=[0]*len(x)
        
        else:
            for i in range(len(x)):
                d[i][len(d[i])-1]-=u*listcont[j][i]/s
                
    return d

Nous obtenons ainsi le gradient de notre fonction pénalisée:

In [50]:
def gradient(x, fonctobj, listcont):
    grad = []
    for i in range(len(x)):
        grad.append(resultat(x[i],contraintelineaire(x, fonctobj, listcont)[i]))
    return grad

Une fois notre programme sans contraintes obtenu, on applique un algorithme de gradient descendant qui nous permet de nous rapprocher du minimum local (dans le domaine de définition de la fonction pénalisée), et c’est ainsi que nous arrivons à obtenir une solution optimale de notre programme (linéaire ou non linéaire)

In [51]:
def methodeDescente(X0, obj, lcont):
    X = X0
  
    gX = gradient(X, obj,lcont)
    
    alpha = Decimal(0.12)
    iterations=0
    while numpy.linalg.norm(gX) > Decimal('0.01'):
        if iterations > 500:
            break
        elif (gX==[0]*len(X0)):
            break
        else:
            alpha*=(1-alpha)
            iterations +=1
            for j in range(len(lcont)):
                for i in range(len(X0)):
                    X[i] -= alpha * gX[i]
                    gX = gradient(X, obj,lcont)
                
    return X

# Etude de la convergence de l'algorithme
La question de convergence n’est pas posée et théorie, mais en pratique c’est une question différente. Elle dépend du pas du gradient utilisé, ainsi que de la puissance de calcul de la machine. Il faut que le pas associé au gradient soit assez petit afin d’éviter un effet "pendule" qui ferait que le programme diverge et ne s’arrête jamais. Nous avons aussi pensé à ajouter un compteur qui retourne un résultat au bout d’un certain nombre d’itérations (ici 500), qui certes réduit la précision du programme, mais garanti au moins l’obtention d’une solution optimale au pas du gradient près.